# Übungsaufgaben 7


## Aufgabe 1: xmlpos (Extraktion und Annotation von TEI-XML)
Schreiben Sie ein Python-Programm `xmlpos`, das aus einer Korpusdatei
den Dokumenteninhalt extrahiert, diesen dann satzweise POS-tagged
und eine XML-Datei folgender Form ausgibt:

```xml
<doc>
  <s id="s-0001">
    <w id="w-0001" pos="DET">Der</w>
	<w id="w-0002" pos="NOUN">Hase</w>
	...
  </s>
  <s id="s-0002">
  ...
</doc>
```

Anmerkungen:
* Verwendenden Sie die [TEI](https://tei-c.org/)-kodierten Korpusdateien des [deutschen Textarchivs](https://www.deutschestextarchiv.de/), z.B:
  * Altman 1890: https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890
  * Brandes 1832: https://www.deutschestextarchiv.de/book/download_xml/brandes_naturlehre03_1832


> _XML-TEI (Text Encoding Initiative): weit verbreitetes XML-Dokumentenformat für die Kodierung von Textkorpora_    
  (Schema mit einheitlichen Elementnamen und Attributen für die editionswissenschaftliche und linguistische Annotation von Texten, u.a. über DTD definiert)

> _DTA (Deutsches Textarchiv): historisches deutsches Korpus_ (linguistisch annotiertes Volltextkorpus deutschsprachiger Texte aus der Zeit um 1600 bis 1900)
      

* Sie können die XML-Dateien auch mit Werkzeugen wie wget oder curl
  (man-Pages!)  herunterladen, oder auch Pythons `urllib` verwenden
* Sie können `nltk` zur Satzsegmentierung und zum POS-Tagging verwenden; alternativ
  dazu können Sie auch _stanza_ (oder _[spacy](https://spacy.io/)_) verwenden
* Stellen Sie sicher, dass der tatsächlichen Text aus den Dokumenten extrahiert wird;
  betrachten Sie hierzu folgendes (valides) XML-Dokument: `<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>` 


* [F-Strings](http://cissandbox.bentley.edu/sandbox/wp-content/uploads/2022-02-10-Documentation-on-f-strings-Updated.pdf)
* [Spacy](https://spacy.io/)
* [DTA](https://www.deutschestextarchiv.de/)
* [TEI](https://github.com/TEIC/TEI)

### Vorgehen:

1. XML-Dokument herunterladen (mit `requests`-Library oder `urllib`-Library)
2. Text (Inhalt von `<text>`-TEI-Element) mit `etree` aus Datei extrahieren:
   - mit XML-Parser über das XML-File iterieren (Namespace des TEI-Wurzelelements berücksichtigen!)
   -  Verwendung von Text-Extractor-Klasse (`TreeExtractor`; alternativ mit Funktionsaufruf: `gather_text`) und Hilfsfunktionen (`append_text`)
3. Satzsegmentierung(mit `NLTK.sent_tokenize()` und POS-Tagging (hier mit `spacy`)
4. Aufbau der neuen XML-Struktur mit `etree` (Satz-Segment- und Wort-POS-Informationen)

#### TEI-XML-Grundstruktur (Altmann-BSP):
```xml
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        ...
    </teiHeader>
    <text>
        <front>
            <titlePage type="halftitle">
                ...
            </titlePage>
        </front>
        <body>
            <div n="1">
                ...
                <p>
                    Seitdem von
                    <hi rendition="#k">Dujardin</hi>
                    die contraktile Substanz oder Sar¬
                    <lb/>
                    kode entdeckt war, hat dieselbe in Bezug auf die Deutung ihres
                    <lb/>
                    ...
                </p>
            </div>
        </body>
        <back>
            ...
        </back>
    </text>
</TEI>
```

### append_text

- setzt getrennte Wörter am Zeilenende (`¬</lb>`) zusammen

In [7]:
def append_text(text, str):
    if not str or str == "":
        return text 
    
    str = str.strip()
    if text == "":
        return str 
    if text[-1] == '¬': # Trennerzeichen in XML-File
        return text[:-1] + str 
    return text + " " + str 

print(f'<<{append_text("abc", "   def ")}>>')
print(f'<<{append_text("unge¬", "legen")}>>')

<<abc def>>
<<ungelegen>>


### TreeExtractor

In [24]:
import xml.etree.ElementTree as ET 

# Achtung: um auch den Text vor dem schließenden Element zu erhalten: tail-Attribut verwenden

# hier zunächst ohne tail:

class TextExtractor:
    def __init__(self, root):
        self.root = root 
        
    def extract(self):
        self.text = ""
        self.extract_rec(self.root) #Iterieren über Kinderknoten mit rekursiver Funktion
        return self.text
        
    def extract_rec(self, node):
        self.text = append_text(self.text, node.text)
        for c in node:  #Iterieren über Kinderknoten
            self.extract_rec(c) #rekursiver Aufruf (für Extraktion des Texts aus Kinder-Knoten)
        #self.text = append_text(self.text, node.tail) #tail für Elemente vor dem schließenden Tag (im BSP: 4 und 5)
        
e = TextExtractor(ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>'))
print(f'<<{e.extract()}>>')

<<1 2>>


In [25]:
import xml.etree.ElementTree as ET 

class TextExtractor:
    def __init__(self, root):
        self.root = root 
        
    def extract(self):
        self.text = ""
        self.extract_rec(self.root) #Iterieren über Kinderknoten mit rekursiver Funktion
        return self.text
        
    def extract_rec(self, node):
        self.text = append_text(self.text, node.text)
        for c in node:  #Iterieren über Kinderknoten
            self.extract_rec(c) #rekursiver Aufruf (für Extraktion des Texts aus Kinder-Knoten)
        self.text = append_text(self.text, node.tail) #tail für Elemente vor dem schließenden Tag (im BSP: 4 und 5)
        
e = TextExtractor(ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>'))
print(f'<<{e.extract()}>>')

<<1 2 3 4 5>>


### Alternative 1: gather_text (Verwendung siehe xmlpos.py)

In [23]:
def gather_text(node, text):
    text = append_text(text, node.text)
    for c in node: #Iterieren über Kinder-Knoten
        text = gather_text(c, text) #rekursiver Aufruf
    #return text
    return append_text(text, node.tail)

root = ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>')
text = gather_text(root, "")
print(f'<<{text}>>')

<<1 2 3 4 5>>


### Alternative 2: gather_text_alt

In [24]:
def gather_text_alt(node):
    return gather_text_rec(node, "")

def gather_text_rec(node, text):
    text = append_text(text, node.text)
    for c in  node:
        text = gather_text_rec(c, text)
    return append_text(text, node.tail)
    
root = ET.fromstring('<a><b>1<c>2<d/>3</c></b>4<lb/>5</a>')
text = gather_text_alt(root)
print(f'<<{text}>>')

<<1 2 3 4 5>>


### Download der XML-Dateien und Verwendung der TextExtractor-Klasse:

In [29]:
import xml.etree.ElementTree as ET 

import urllib.request
url = "https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890"
# url = "https://www.deutschestextarchiv.de/book/download_xml/brandes_naturlehre03_1832"


with urllib.request.urlopen(url) as f:
    root = ET.fromstring(f.read())

# TEI-Namespace (Namespace als Präfix der Elementnamen, z.B. http://www.tei-c.org/ns/1.0:element)
ns = {'tei': 'http://www.tei-c.org/ns/1.0'} #Namespace-Dictionary mit Abkürzungen

# Extraktion mit <tei:text> als Wurzelknoten (d.h. ohne Metadaten in <teiHeader>):
e = TextExtractor(root.find('tei:text', ns))
text = e.extract()
print(text[1002:1100])

zu sichern. Das Bewusstsein, dass uns hier die Grundprobleme der Biologie berühren, wird es hoffen


### Download des [Spacy](https://spacy.io) Modells

In [ ]:
%%bash
#python -m spacy download de_core_news_sm

### Importe

In [8]:
import sys
#!{sys.executable} -m pip install --upgrade pip spacy
import spacy
#!{sys.executable} -m pip install --upgrade pip nltk
from nltk import sent_tokenize
import xml.dom.minidom

### POS-Tagging und Aufbau des XML-Baums

In [18]:
nlp = spacy.load('de_core_news_sm')
out = ET.Element('doc') #für Erzeugung XML-File
sid = 1 # sentence id
wid = 1 # word id

#Satzsegmentierung mit NLTK-Methode:
sents = sent_tokenize(text)

#POS-Tagging mit spaCy und Erzeugung XML-File:
for sent in sents:
    stag = ET.SubElement(out, 's') #Erzeugung von s-Element
    stag.attrib = {'id': f's-{sid:05d}'}
    sid += 1
    tokens = nlp(sent) #Analyse mit spaCy
    for token in tokens: #Erzeugen von w-Elementen (pro Satz)
        wtag = ET.SubElement(stag, 'w')
        wtag.text = token.text
        wtag.attrib = {'pos': token.pos_, 'id': f'w-{wid:05d}'}
        wid += 1

dom = xml.dom.minidom.parseString(ET.tostring(out))
print(dom.toprettyxml()[:1500])

<?xml version="1.0" ?>
<doc>
	<s id="s-00001">
		<w id="w-00001" pos="DET">DIE</w>
		<w id="w-00002" pos="NOUN">ELEMENTARORGANISMEN</w>
		<w id="w-00003" pos="PROPN">BEZIEHUNGEN</w>
		<w id="w-00004" pos="ADP">ZU</w>
		<w id="w-00005" pos="PROPN">DEN</w>
		<w id="w-00006" pos="PROPN">ZELLEN</w>
		<w id="w-00007" pos="ADP">VON</w>
		<w id="w-00008" pos="SPACE"> </w>
		<w id="w-00009" pos="PROPN">RICHARD</w>
		<w id="w-00010" pos="PROPN">ALTMANN</w>
		<w id="w-00011" pos="PROPN">MIT</w>
		<w id="w-00012" pos="PROPN">ZWEI</w>
		<w id="w-00013" pos="PROPN">ABBILDUNGEN</w>
		<w id="w-00014" pos="ADP">IM</w>
		<w id="w-00015" pos="PROPN">TEXT</w>
		<w id="w-00016" pos="CCONJ">UND</w>
		<w id="w-00017" pos="PROPN">XXI</w>
		<w id="w-00018" pos="PROPN">TAFELN</w>
		<w id="w-00019" pos="PUNCT">.</w>
	</s>
	<s id="s-00002">
		<w id="w-00020" pos="PROPN">LEIPZIG</w>
		<w id="w-00021" pos="PUNCT">,</w>
		<w id="w-00022" pos="PROPN">VERLAG</w>
		<w id="w-00023" pos="PROPN">VON</w>
		<w id="w-00024"

### Usage von xmlpos.py (mit Ausgabe in Datei):

#### Einlesen aus Datei:

In [30]:
%%bash
cat altmann_elementarorganismen_1890.TEI-P5.xml | python3 xmlpos.py > altmann_elementarorganismen_1890.xml

In [37]:
%%bash
xmllint --format altmann_elementarorganismen_1890.xml | head -25

<?xml version="1.0"?>
<doc>
  <s id="s-1">
    <w id="w-1" pos="DET">DIE</w>
    <w id="w-2" pos="SPACE"> </w>
    <w id="w-3" pos="PROPN">ELEMENTARORGANISMEN</w>
    <w id="w-4" pos="CCONJ">UND</w>
    <w id="w-5" pos="PROPN">IHRE</w>
    <w id="w-6" pos="SPACE"> </w>
    <w id="w-7" pos="PROPN">BEZIEHUNGEN</w>
    <w id="w-8" pos="ADP">ZU</w>
    <w id="w-9" pos="PROPN">DEN</w>
    <w id="w-10" pos="PROPN">ZELLEN</w>
    <w id="w-11" pos="PUNCT">.</w>
  </s>
  <s id="s-2">
    <w id="w-12" pos="ADP">VON</w>
    <w id="w-13" pos="SPACE">  </w>
    <w id="w-14" pos="PROPN">RICHARD</w>
    <w id="w-15" pos="PROPN">ALTMANN</w>
    <w id="w-16" pos="PUNCT">.</w>
  </s>
  <s id="s-3">
    <w id="w-17" pos="PROPN">MIT</w>
    <w id="w-18" pos="PROPN">ZWEI</w>


#### Herunterladen der XML-Datei mit curl:

In [26]:
%%bash
curl https://www.deutschestextarchiv.de/book/download_xml/altmann_elementarorganismen_1890 | python3 xmlpos.py > altmann_elementarorganismen_1890.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  385k  100  385k    0     0   202k      0  0:00:01  0:00:01 --:--:--  202k


---
## Aufgabe 2: xmltcf (Verarbeitung von annotierten XML-Dateien im TCF-Format)

Im deutschen Textarchiv findet man auch annotierte Daten zu den
entsprechenden Dokumenten.  Laden Sie eine solche lemmatisierte und
annotierte XML-Datei im [TCF-Format](https://weblicht.sfs.uni-tuebingen.de/weblichtwiki/index.php/The_TCF_Format) (Text Corpus Format)
herunter (z.B. Altmann: https://www.deutschestextarchiv.de/book/download_fulltcf/16299)
und schreiben Sie ein Programm `xmltcf`, das eine zeilenweise die Sätze
ausgibt, wobei die Token entweder mit ihren Lemmata oder ihren Tags
annotiert sind:
```bash
$ ./xmltcf.py file.xml
Ich/ich gehe/gehen ...
$ ./xmltcf.py --tags file.xml
Der/DET Mann/NOUN ...
```


### Vorgehen:

- Iterieren über Sätze (`<sentences>`) und Tokens (`<tokens>`) des TCF-XML-Files
- Lookup von POS (`<POStags>`) und Lemmata (`<lemmas>`) im XML über `tokenIDs`

#### TCF-XML-Grundstruktur:
```xml
<D-Spin xmlns="http://www.dspin.de/data" version="0.4">
    <MetaData xmlns="http://www.dspin.de/data/metadata">
        ...
    </MetaData>
    <TextCorpus  xmlns="http://www.dspin.de/data/textcorpus" lang="de">
        <tokens>
            <token ID="w1">zwei</token>
            <token ID="w2">Tests</token>
        </tokens>
        <sentences>
            <sentence ID="s1" tokenIDs="w1 w2"/>
        </sentences>
        <lemmas>
            <lemma tokenIDs="w1">zwei</lemma>
            <lemma tokenIDs="w2">Test</lemma>
        </lemmas>
        <POStags tagset="stts">
            <tag tokenIDs="w1">ART</tag>
            <tag tokenIDs="w2">NN</tag>
        </POStags>
        <orthography>
            ...
        </orthography>
    </TextCorpus>
</D-Spin>
```

#### xmltcf.py:

In [ ]:
#!/usr/bin/env python3

import urllib.request
import xml.etree.ElementTree as ET
import argparse

# Token-Daten-Klasse: speichert Token, ID, POS-Tag und Lemma
class Token:
    def __init__(self, word, id, tag=None, lemma=None):
        self.word = word
        self.id = id
        self.tag = tag
        self.lemma = lemma

# url = 'https://www.deutschestextarchiv.de/book/download_fulltcf/16299'

def parse_xml(url, tags):
    with urllib.request.urlopen(url) as f:
        root = ET.fromstring(f.read())

#    with open(name, encoding='utf-8') as f:
#        root = ET.fromstring(f.read())
        
    #Namespace-Dictionary (TextCorpus-Namespace):
    ns = {'tc': "http://www.dspin.de/data/textcorpus"} 
    
    #1. Iterieren über Token-Elemente im TextCorpus-Knoten, Generierung lex-Dictionary mit Tokens+IDs:
    # (Verwendung von Token-Datenklasse (oben definiert) in Dictionary-Comprehension)
    lex = {t.attrib["ID"]: Token(t.text, t.attrib["ID"]) for t in root.find('tc:TextCorpus', ns).find('tc:tokens', ns)}

    #2. Iterieren über Lemmas- und POSTags-Elemente und Ergänzung in lex-Dictionary (über tokenIDs):
    for lemma in root.find('tc:TextCorpus', ns).find('tc:lemmas', ns ):
        token = lex[lemma.attrib["tokenIDs"]]
        token.lemma = lemma.text
    for tag in root.find('tc:TextCorpus', ns ).find('tc:POStags', ns):
        token = lex[tag.attrib["tokenIDs"]]
        token.tag = tag.text
        
    # lex-Dictionary enthält für jede Token-ID: word, id, pos-tag, lemma

    #3. Iterieren über sentences-Knoten und satzweises Erzeugen von "Token/Tag"- bzw. "Token/Lemma"-Strings: 
    # (Lookup in lex-Dictionary über tokenIDs)
    #(in Abhängigkeit von argparse-Option -t = tags bei Ausführung):
    for sent in root.find('tc:TextCorpus', ns).find('tc:sentences', ns):
        if tags:
            print(" ".join([lex[id].word + "/" + lex[id].tag for id in sent.attrib["tokenIDs"].split(" ")]))
        else:
            print(" ".join([lex[id].word + "/" + lex[id].lemma for id in sent.attrib["tokenIDs"].split(" ")]))


# Argumente für Programmaufruf über Kommandozeile (inklusive Help-Option):
parser = argparse.ArgumentParser(description='write tokens with their tags or lemmas from tcf files')
parser.add_argument('-t', '--tags', help='output tokens with their tag instead of their lemma', action="store_true")
parser.add_argument('urls', help='list of urls to operate on', nargs='*')

args = parser.parse_args()
for url in args.urls:
    parse_xml(url, args.tags)


#### Usage:

In [9]:
%%bash
./xmltcf.py -h

usage: xmltcf.py [-h] [-t] [urls [urls ...]]

write tokens with their tags or lemmas from tcf files

positional arguments:
  urls        list of urls to operate on

optional arguments:
  -h, --help  show this help message and exit
  -t, --tags  output tokens with their tag instead of their lemma


#### Ausgabe Sätze mit POS-Tags:

In [22]:
%%bash
./xmltcf.py -t https://www.deutschestextarchiv.de/book/download_fulltcf/16299 > sents.txt

In [25]:
%%bash
sed 10q sents.txt

DIE/ART ELEMENTARORGANISMEN/NN UND/KON IHRE/ADJA BEZIEHUNGEN/NN ZU/APPR DEN/ART ZELLEN/NN ./$. VON/NN RICHARD/NE ALTMANN/NE ./$.
MIT/NE ZWEI/ADJA ABBILDUNGEN/NN IM/APPR TEXT/NE UND/KON XXI/NE TAFELN./NE LEIPZIG/NE ,/$, VERLAG/NN VON/NE VEIT/NE &/$( COMP./NE 1890/CARD ./$.
DIE/ART ELEMENTARORGANISMEN/NN UND/KON IHRE/ADJA BEZIEHUNGEN/NN ZU/APPR DEN/ART ZELLEN/NN ./$. VON/NN RICHARD/NE ALTMANN/NE ./$.
MIT/NE ZWEI/ADJA ABBILDUNGEN/NN IM/APPR TEXT/NE UND/KON XXI/NE TAFELN./NE LEIPZIG/NE ,/$, VERLAG/NN VON/NE VEIT/NE &/$( COMP./NE 1890/CARD ./$.
HERRN/NN WILHELM/NE HIS/NE IN/APPR DANKBARER/ADJA VEREHRUNG/NN GEWIDMET/VVPP VOM/ADJA VERFASSER/NN ./$.
Vorbemerkung/NN ./$.
Die/ART nachfolgenden/ADJA Capitel/NN enthalten/VVFIN im/APPRART Wesentlichen/NN eine/ART theils/ADV erweiterte/ADJA ,/$, theils/ADV verkürzte/ADJA Zusammenstellung/NN derjenigen/PDS Abhandlungen/NN ,/$, welche/PRELS bisher/ADV von/APPR mir/PPER über/APPR die/ART Zellengranula/NN veröffentlicht/VVPP worden/VAPP sind/VAFIN ./$.


#### Ausgabe Sätze mit Lemmata:

In [26]:
%%bash
./xmltcf.py https://www.deutschestextarchiv.de/book/download_fulltcf/16299 > sents_lemmata.txt

In [27]:
%%bash
sed 10q sents_lemmata.txt

DIE/d ELEMENTARORGANISMEN/Elementarorganismus UND/und IHRE/Ihre BEZIEHUNGEN/Beziehung ZU/zu DEN/d ZELLEN/Zelle ./. VON/VON RICHARD/Richard ALTMANN/Altmann ./.
MIT/Massachussetts_Institute_Of_Technology ZWEI/zwei ABBILDUNGEN/Abbildung IM/IM TEXT/Text UND/und XXI/Xxi TAFELN./TAFELN. LEIPZIG/Leipzig ,/, VERLAG/Verlag VON/VON VEIT/Veit &/& COMP./COMP. 1890/1890 ./.
DIE/d ELEMENTARORGANISMEN/Elementarorganismus UND/und IHRE/Ihre BEZIEHUNGEN/Beziehung ZU/zu DEN/d ZELLEN/Zelle ./. VON/VON RICHARD/Richard ALTMANN/Altmann ./.
MIT/Massachussetts_Institute_Of_Technology ZWEI/zwei ABBILDUNGEN/Abbildung IM/IM TEXT/Text UND/und XXI/Xxi TAFELN./TAFELN. LEIPZIG/Leipzig ,/, VERLAG/Verlag VON/VON VEIT/Veit &/& COMP./COMP. 1890/1890 ./.
HERRN/Herr WILHELM/Wilhelm HIS/His IN/in DANKBARER/dankbar VEREHRUNG/Verehrung GEWIDMET/widmen VOM/vom VERFASSER/Verfasser ./.
Vorbemerkung/Vorbemerkung ./.
Die/d nachfolgenden/nachfolgend Capitel/Kapitel enthalten/enthalten im/im Wesentlichen/Wesentliche eine/eine theils